In [1]:
import subprocess
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification, pipeline
from datasets import load_dataset, Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import logging
import numpy as np
from trl import DPOTrainer, DPOConfig

# Load environment variables from /etc/network_turbo
result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

# Set the model path of qwen sft AND sentiment model
LM_MODEL = "august66/qwen2-sft-final"
SENTIMENT_MODEL = "siebert/sentiment-roberta-large-english"
N_PREFIX_TOKENS = 5


#load dataset
dataset_test = load_dataset("stanfordnlp/imdb", split="test")
def prompt_completion_preprocess(example):
    words = example['text'].split()
    prompt = ' '.join(words[:N_PREFIX_TOKENS])
    completion = ' '.join(words[N_PREFIX_TOKENS:])
    return {'prompt': prompt, 'completion': completion}
dataset_test = dataset_test.map(prompt_completion_preprocess, remove_columns=['text', 'label'])






In [ ]:
qwen_sft_model = AutoModelForCausalLM.from_pretrained(LM_MODEL)
qwen_sft_tokenizer = AutoTokenizer.from_pretrained(LM_MODEL)
qwen_sft_tokenizer.padding_side = "left"
qwen_sft_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
pipe = pipeline(
    'text-generation',
    model = qwen_sft_model,
    tokenizer = qwen_sft_tokenizer,
    device_map = 'auto'
)
prompts = dataset_test['prompt']
generated_completions = pipe(
    prompts,
    max_new_tokens = 128,
    do_sample = True,
    truncation = True,
    padding = True,
    top_p = 0.95,
    temperature = 1,
    num_return_sequences = 2,
    batch_size = 128,
)


Device set to use cuda:0
